In [1]:

#overfitting / underfitting 을 조절하기 위해서 할 수 있는 방법
#data를 추가하기 / early stop 사용하기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [5]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
EPOCHS = 50
BATCH_SIZE = 64
# Compose 함수를 호출해서 사용한 randomhorizontalflip함수는 이미지를 무작위로 수평 뒤집기를 한다.
# 이 함수만으로 이미지데이터가 2배가 된다.
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',train = True,download=True,transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))])),batch_size=BATCH_SIZE,shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',train = False,download=True,transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.1307),(0.3081))])),batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
#drop out
#데이터뿐만 아니라 모델에 직접 영향을 줘서 과적합을 해결하는 드롭아웃 방법이 있다. 학습 도중 신경망의 일부를 사용하지 않는 방법.
# 검증과 테스트 단계에서는 모든 뉴런을 사용하지만 학습뉴런을 몇개를 죽임으로서 다른 뉴런들의 가중치가 고이는 것을 방지할 수 있음
class Net(nn.Module):
    def __init__(self,dropout_p=0.2):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10)
        #드롭아웃 확률 설정
        self.dropout_p = dropout_p
    def forward(self,x):
        x = x.view(-1,784)
        x = F.relu(self.fc1(x))
        #드롭아웃 추가
        x = F.dropout(x,training=self.training,p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x,training=self.training,p=self.dropout_p)
        x = self.fc3(x)
        return x
    
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(),lr = 0.01)

def train(model,train_loader,optimizer):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(DEVICE),target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        
def evaluate(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(DEVICE),target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output,target,reduction = 'sum').item()
            pred = output.max(1,keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_acc = 100*correct/len(test_loader.dataset)
    return test_loss,test_acc

for epoch in range(EPOCHS):
    train(model,train_loader,optimizer)
    test_loss,test_acc = evaluate(model,test_loader)
    print('epoch',epoch,'loss',test_loss,'acc',test_acc)
    
#이 코드를 통해서 데이터의 추가와 dropout으로 인해서 정확도가 기존보다 상승하는 것을 볼 수 있다.

epoch 0 loss 0.5562495931625366 acc 82.45
epoch 1 loss 0.4276498867034912 acc 86.67
epoch 2 loss 0.35771887044906614 acc 88.54
epoch 3 loss 0.30509262845516205 acc 90.31
epoch 4 loss 0.2637909323215485 acc 91.83
epoch 5 loss 0.2349925431907177 acc 92.63
epoch 6 loss 0.21358354201316834 acc 93.38
epoch 7 loss 0.19619987268447875 acc 93.98
epoch 8 loss 0.18368469135761262 acc 94.15
epoch 9 loss 0.17183201806545256 acc 94.57
epoch 10 loss 0.16419848544597626 acc 94.9
